<a href="https://colab.research.google.com/github/GrigoreGeorgeAlexandru/Colab-projects/blob/main/Laborator3_D_Word_Count_stud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3a038b62323e3153439e40d5565d78c1cfa1d5916602c2bcaa345d1fc4469ee6
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Numărarea cuvintelor

### Numărarea aparițiilor cuvintelor într-un text este, de obicei, primul exercițiu pentru aplicarea metodei map-reduce.

## Problema
**Input:** Un fișier text ce constă din cuvinte separate prin spații.  
**Output:** O listă de cuvinte și numărul lor de apariții. Lista va fi sortată descrescător după numărul de apariții.

Vom folosi fișierul ce conține cartea "Moby Dick" ca input.

In [ ]:
from pyspark import SparkContext
#Porniti SparkContext cu 4 core-uri
sc = SparkContext(master="local[4]")

### Definiți o funcție pentru afișarea corespunzătoare a planului de execuție

In [ ]:
def pretty_print_plan(rdd):
    for x in rdd.toDebugString().decode().split('\n'):
        print(x)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Utilizați metoda `textFile()` pentru a citi textul din fișier

In [ ]:
%%time
text_file = sc.textFile("/content/drive/MyDrive/Data/Moby-Dick.txt")

CPU times: user 10.3 ms, sys: 4.18 ms, total: 14.5 ms
Wall time: 1.17 s


In [ ]:
type(text_file)

pyspark.rdd.RDD

## Pașii pentru numărarea cuvintelor

* se separă elementele unei linii în funcție de separatori (spații).
* se mapează `word` la `(word,1)`
* se numără aparițiile fiecărui cuvânt.

In [ ]:
%%time
words = text_file.flatMap(lambda line: line.split(" "))
#obtineti cuvintele nevide
not_empty = words.filter(lambda x : len(x)>0)
#mapati word -> (word, 1)
key_values=  not_empty.map(lambda x: (x,1))
#numarati cuvintele, cu ajutorul reduceByKey, cu o functie lambda ca argument
counts= key_values.reduceByKey(lambda x,y : x+y)

CPU times: user 12.3 ms, sys: 4.5 ms, total: 16.8 ms
Wall time: 214 ms


### flatMap()
In linia de cod:
```python
words =     text_file.flatMap(lambda line: line.split(" "))
```
este utilizat `flatMap` în loc de `map`.

Motivul este acela că operația `line.split(" ")` generează o **listă** de string-uri, deci dacă am folosi `map` rezultatul ar fi un RDD de liste de cuvinte, nu un RDD de cuvinte.

Diferența dintre `map` și `flatMap` este aceea că `flatMap` asteaptă câte o listă care provine din rezultatul câte unui map și **concatenează** listele spre a forma RDD-ul.

## Planul de execuție
În ultima celulă am definit planul de execuție, dar nu l-am executat.

* Pregătirea planului a durat ~100ms, un interval de timp netrivial, dar mai scurt decat timpul necesar execuției.
* Vom analiza planul de execuție.

### Detaliile planului de execuție
Pentru a determina cărui RDD îi corespunde fiecare pas din plan, vom afișa planul de execuție pentru fiecare dintre RDD-uri.  

Se va observa că planurile de execuție pentru `words`, `not_empty` și `key_values` sunt aceleași.

In [ ]:
pretty_print_plan(text_file)

(2) /content/drive/MyDrive/Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /content/drive/MyDrive/Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [ ]:
pretty_print_plan(words)

(2) PythonRDD[6] at RDD at PythonRDD.scala:53 []
 |  /content/drive/MyDrive/Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /content/drive/MyDrive/Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [ ]:
pretty_print_plan(not_empty)

(2) PythonRDD[7] at RDD at PythonRDD.scala:53 []
 |  /content/drive/MyDrive/Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /content/drive/MyDrive/Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [ ]:
pretty_print_plan(key_values)

(2) PythonRDD[8] at RDD at PythonRDD.scala:53 []
 |  /content/drive/MyDrive/Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /content/drive/MyDrive/Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


In [ ]:
pretty_print_plan(counts)

(2) PythonRDD[9] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[5] at mapPartitions at PythonRDD.scala:160 []
 |  ShuffledRDD[4] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[3] at reduceByKey at <timed exec>:7 []
    |  PythonRDD[2] at reduceByKey at <timed exec>:7 []
    |  /content/drive/MyDrive/Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []
    |  /content/drive/MyDrive/Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []


| Execution plan   | RDD |  Comments |
| :---------------------------------------------------------------- | :------------: | :--- |
|`(2)_PythonRDD[6] at RDD at PythonRDD.scala:48 []`| **counts** | Final RDD|
|`_/__MapPartitionsRDD[5] at mapPartitions at PythonRDD.scala:436 []`| **---"---** |
|`_/__ShuffledRDD[4] at partitionBy at NativeMethodAccessorImpl.java:0 [`| **---"---** | RDD is partitioned by key |
|`_+-(2)_PairwiseRDD[3] at reduceByKey at <timed exec>:4 []`| **---"---** | Perform mapByKey |
|`____/__PythonRDD[2] at reduceByKey at <timed exec>:4 []`| **words, not_empty, key_values** | The result of  partitioning into words|
| | |  removing empties, and making into (word,1) pairs|
|`____/__../../Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at Nat`| **text_file** | The partitioned text |
|`____/__../../Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMeth`| **---"---** | The text source |

## Execuția
Numărăm aparițiile fiecărui cuvânt. Abia acum modelul cu execuție _lazy_ efectuează operațiile, motiv pentru care execuția durează mai mult.

In [ ]:
print(counts.collect())

[('The', 549), ('Project', 79), ('EBook', 1), ('of', 6587), ('Moby', 79), ('is', 1586), ('use', 35), ('anyone', 5), ('anywhere', 11), ('at', 1227), ('no', 447), ('restrictions', 2), ('whatsoever.', 5), ('may', 223), ('it,', 237), ('give', 68), ('away', 117), ('re-use', 2), ('this', 1169), ('online', 4), ('www.gutenberg.org', 2), ('Author:', 1), ('Last', 1), ('January', 1), ('3,', 2), ('Posting', 1), ('Date:', 2), ('#2701]', 1), ('June,', 3), ('Language:', 1), ('English', 42), ('***', 6), ('OF', 59), ('THIS', 12), ('GUTENBERG', 3), ('MOBY', 3), ('DICK;', 3), ('Lazarus', 6), ('Original', 1), ('combination', 2), ('now-defunct', 1), ('project', 3), ('version', 3), ('are', 586), ('University', 1), ('Adelaide', 1), ('preserving', 3), ('version.', 1), ('resulting', 3), ('was', 1566), ('compared', 9), ('public', 13), ('domain', 8), ('89,', 1), ('we', 395), ('L', 1), ('symbol', 10), ('currency.', 1), ('ETYMOLOGY.', 1), ('(Supplied', 2), ('Consumptive', 1), ('Usher', 1), ('School)', 1), ('pale',

In [ ]:
%%time
## Run #1
Count=counts.count()  # Count = the number of different words
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y) #
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

PythonRDD[9] at RDD at PythonRDD.scala:53
Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 84.3 ms, sys: 9.76 ms, total: 94 ms
Wall time: 3.54 s


### Amortizarea
Atunci când aceleași comenzi sunt efectuate în mod repetat pe aceleași date, timpul pentru execuțiile ulterioare tinde să descrească.

Celulele de mai jos sunt identice celei anterioare, cu o excepție la `Run #3`

Observăm că `Run #2` durează mai puțin decât `Run #1` cu toate că nu a fost aplicată explicit metoda `cache()`. Motivul este acela că Spark pune în cache (materializează) `key_values`, înainte de a executa `reduceByKey()`. Efectuarea operației _reduceByKey_ necesită o amestecare (_shuffle_), iar un _shuffle_ necesită materializarea RDD-ului de intrare. Prin urmare, observăm ca punerea în cache (caching-ul) are uneori loc chiar dacă nu a fost solicitat explicit.

In [ ]:
%%time
## Run #2
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 23.1 ms, sys: 1.94 ms, total: 25 ms
Wall time: 1.39 s


### Caching Explicit
În `Run #3` solicităm explicit punerea în _cache_ a lui `counts`. Aceasta va reduce timpul la următoarea execuție `Run #4`, însă nu în mod semnificativ.

In [ ]:
%%time
## Run #3, cache
Count=counts.cache().count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 29.4 ms, sys: 2.38 ms, total: 31.8 ms
Wall time: 2.38 s


In [ ]:
%%time
#Run #4
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 19.3 ms, sys: 3.89 ms, total: 23.2 ms
Wall time: 807 ms


In [ ]:
%%time
#Run #5
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 15.2 ms, sys: 6.92 ms, total: 22.1 ms
Wall time: 1.01 s


In [ ]:
sc.stop()